# Ethical Zeta Function - Deep Zero Analysis

This notebook performs advanced analysis of ethical zeta function zeros, including:
- Critical line analysis (distance from Re(s) = 1/2)
- Zero density and distribution
- Pair correlation functions
- Spacing distributions
- Comparison with classical Riemann zeta patterns


In [1]:
import sys
import os
from pathlib import Path

# Robust path setup
def setup_paths():
    current_dir = Path(os.getcwd())
    if current_dir.name == 'notebooks':
        simulation_dir = str(current_dir.parent)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    elif current_dir.name == 'simulation':
        simulation_dir = str(current_dir)
        if simulation_dir not in sys.path:
            sys.path.insert(0, simulation_dir)
        return simulation_dir
    for parent in current_dir.parents:
        if parent.name == 'simulation':
            simulation_dir = str(parent)
            if simulation_dir not in sys.path:
                sys.path.insert(0, simulation_dir)
            return simulation_dir
    for path in ['..', '../simulation', 'simulation']:
        abs_path = os.path.abspath(path)
        if abs_path not in sys.path:
            sys.path.insert(0, abs_path)
    return None

setup_paths()

import numpy as np
import matplotlib.pyplot as plt

from core.action_space import generate_world
from core.judgement_system import BiasedJudge, evaluate_judgement
from core.ethical_primes import select_ethical_primes
from analysis.zeta_function import build_m_sequence, find_approximate_zeros, ethical_zeta_sum
from analysis.zeta_zeros_analysis import (
    refine_zero_newton_raphson,
    analyze_critical_line_proximity,
    compute_zero_density,
    compute_pair_correlation,
    compute_spacing_distribution,
    compare_with_classical_zeta_patterns
)
from visualization.plots import setup_paper_style

setup_paper_style()
np.random.seed(42)


## Generate Data and Find Zeros


In [2]:
# Generate data
num_actions = 2000
actions = generate_world(num_actions=num_actions, random_seed=42)
judge = BiasedJudge(bias_strength=0.2)
evaluate_judgement(actions, judge, tau=0.3)
primes = select_ethical_primes(actions, importance_quantile=0.9)

print(f"Generated {num_actions} actions")
print(f"Found {len(primes)} ethical primes")

# Build m sequence
m = build_m_sequence(primes, X_max=200, mode='count')

# Find zeros with refinement
real_range = (0.3, 0.7)
imag_range = (0, 30)
zeros = find_approximate_zeros(m, real_range=real_range, imag_range=imag_range, 
                               grid_size=40, threshold=0.1, refine=True)

print(f"\nFound {len(zeros)} approximate zeros")
if len(zeros) > 0:
    print(f"Zero range: Re(s) ∈ [{min(z.real for z in zeros):.3f}, {max(z.real for z in zeros):.3f}]")
    print(f"            Im(s) ∈ [{min(z.imag for z in zeros):.3f}, {max(z.imag for z in zeros):.3f}]")


Generated 2000 actions
Found 23 ethical primes

Found 0 approximate zeros


## Critical Line Analysis

Analyze proximity to the critical line Re(s) = 1/2.


In [3]:
if len(zeros) > 0:
    critical_analysis = analyze_critical_line_proximity(zeros, critical_line=0.5)
    
    print("Critical Line Analysis:")
    print("=" * 60)
    print(f"Mean distance from Re(s)=0.5: {critical_analysis['mean_distance']:.4f}")
    print(f"Median distance: {critical_analysis['median_distance']:.4f}")
    print(f"Min distance: {critical_analysis['min_distance']:.4f}")
    print(f"Max distance: {critical_analysis['max_distance']:.4f}")
    print(f"\nZeros within 0.1 of critical line: {critical_analysis['within_0.1']}/{len(zeros)} ({critical_analysis['fraction_within_0.1']:.1%})")
    print(f"Zeros within 0.05: {critical_analysis['within_0.05']}/{len(zeros)} ({critical_analysis['fraction_within_0.05']:.1%})")
    print(f"Zeros within 0.01: {critical_analysis['within_0.01']}/{len(zeros)} ({critical_analysis['fraction_within_0.01']:.1%})")
    
    # Visualize
    distances = [abs(z.real - 0.5) for z in zeros]
    plt.figure(figsize=(10, 6))
    plt.hist(distances, bins=20, edgecolor='black', alpha=0.7)
    plt.axvline(critical_analysis['mean_distance'], color='r', linestyle='--', 
               label=f"Mean: {critical_analysis['mean_distance']:.3f}")
    plt.axvline(0.1, color='g', linestyle='--', alpha=0.5, label='0.1 threshold')
    plt.xlabel('Distance from Critical Line |Re(s) - 0.5|')
    plt.ylabel('Frequency')
    plt.title('Distribution of Zero Distances from Critical Line')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('../output/figures/07_critical_line_analysis.pdf', dpi=300)
    plt.show()
else:
    print("No zeros found for analysis")


No zeros found for analysis


## Zero Density and Pair Correlation


In [4]:
if len(zeros) > 0:
    # Zero density
    density_stats = compute_zero_density(zeros, real_range, imag_range)
    print("Zero Density Analysis:")
    print("=" * 60)
    print(f"Total zeros: {density_stats['total_zeros']}")
    print(f"Search area: {density_stats['area']:.2f}")
    print(f"Density: {density_stats['density']:.4f} zeros per unit area")
    
    # Pair correlation
    distances, correlation = compute_pair_correlation(zeros, max_distance=5.0, num_bins=30)
    
    if len(distances) > 0:
        plt.figure(figsize=(12, 5))
        
        # Plot 1: Pair correlation
        ax1 = plt.subplot(1, 2, 1)
        ax1.plot(distances, correlation, 'b-', linewidth=2)
        ax1.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Uniform (expected)')
        ax1.set_xlabel('Distance between zeros')
        ax1.set_ylabel('Pair Correlation')
        ax1.set_title('Pair Correlation Function')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Spacing distribution
        ax2 = plt.subplot(1, 2, 2)
        spacing_stats = compute_spacing_distribution(zeros, sort_by='imaginary')
        if len(spacing_stats['spacings']) > 0:
            ax2.hist(spacing_stats['spacings'], bins=20, edgecolor='black', alpha=0.7)
            ax2.axvline(spacing_stats['mean_spacing'], color='r', linestyle='--',
                       label=f"Mean: {spacing_stats['mean_spacing']:.3f}")
            ax2.set_xlabel('Spacing between consecutive zeros')
            ax2.set_ylabel('Frequency')
            ax2.set_title('Zero Spacing Distribution')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('../output/figures/07_zero_correlation.pdf', dpi=300)
        plt.show()
        
        print(f"\nSpacing Statistics:")
        print(f"  Mean spacing: {spacing_stats['mean_spacing']:.4f}")
        print(f"  Median spacing: {spacing_stats['median_spacing']:.4f}")
        print(f"  Std spacing: {spacing_stats['std_spacing']:.4f}")


## Comparison with Classical Riemann Zeta Patterns


In [5]:
if len(zeros) > 0:
    comparison = compare_with_classical_zeta_patterns(zeros)
    
    print("Comparison with Classical ζ(s) Patterns:")
    print("=" * 60)
    print(f"Critical line clustering: {comparison['critical_line_clustering']}")
    print(f"Spacing regularity: {comparison['spacing_regularity']}")
    print(f"Similarity score: {comparison['similarity_score']:.3f}")
    
    if 'spacing_cv' in comparison and not np.isnan(comparison['spacing_cv']):
        print(f"Spacing coefficient of variation: {comparison['spacing_cv']:.4f}")
    
    print("\nInterpretation:")
    if comparison['similarity_score'] > 0.7:
        print("  Strong similarity to classical ζ(s) patterns detected!")
        print("  This suggests deep structural regularity in ethical prime distribution.")
    elif comparison['similarity_score'] > 0.4:
        print("  Moderate similarity - some patterns align with classical zeta function.")
    else:
        print("  Limited similarity - ethical zeta may have distinct properties.")
